# Mining PALSAR Analysis

## Objectives
- Lineament extraction and structural analysis
- Integration with SRTM 30 DEM
- Temporal comparison (2001-2006 vs 2023-2024)

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m


In [59]:
pip install geemap earthengine-api numpy

Note: you may need to restart the kernel to use updated packages.


In [60]:
# Import required libraries
import geemap
import ee
import numpy as np
from datetime import datetime

# Initialize Earth Engine with Project ID
# IMPORTANT: Replace 'your-project-id' with your actual Google Cloud Project ID
# To find your Project ID:
# 1. Go to https://console.cloud.google.com/
# 2. Click on the project dropdown at the top
# 3. Copy the Project ID (e.g., 'ee-your-username' or 'mineral-exploration-456')

PROJECT_ID = 'ee-okwaretom12'  # <-- REPLACE THIS WITH YOUR PROJECT ID

try:
    # Try to initialize with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print("Earth Engine not initialized. Starting authentication...")
    print("Please follow the authentication steps:")
    print("1. A browser window will open")
    print("2. Sign in with your Google account")
    print("3. Grant permissions to Earth Engine")
    print("4. Copy the authorization code and paste it when prompted")
    
    # Authenticate (this will open a browser for first-time users)
    ee.Authenticate()
    
    # Initialize after authentication with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized successfully with project: {PROJECT_ID}!")

# Initialize geemap
Map = geemap.Map()
print("Libraries imported and geemap initialized successfully!")


Earth Engine initialized with project: ee-okwaretom12
Libraries imported and geemap initialized successfully!


## Define Study Area


In [61]:
# Define study area coordinates (WGS1984)
# Convert DMS to decimal degrees
coords = [
    [29.575, 4.766667],  # Point 1: 29°34'30"E, 04°46'0"N
    [29.279167, 4.766667],  # Point 2: 29°16'45"E, 04°46'0"N
    [29.279167, 5.091667],  # Point 3: 29°16'45"E, 05°05'30"N
    [29.575, 5.091667],  # Point 4: 29°34'30"E, 05°05'30"N
]

# Create polygon
study_area = ee.Geometry.Polygon([coords])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

# Print study area info (without .getInfo() to avoid blocking)
print("Study area created with coordinates:")
print("  Point 1: 29°34'30\"E, 04°46'0\"N")
print("  Point 2: 29°16'45\"E, 04°46'0\"N")
print("  Point 3: 29°16'45\"E, 05°05'30\"N")
print("  Point 4: 29°34'30\"E, 05°05'30\"N")
print("  Buffer: 20m")

# Add to map
Map.addLayer(study_area_buffered, {'color': 'red'}, 'Study Area (20m buffer)')
Map.centerObject(study_area_buffered, 10)
Map


Study area created with coordinates:
  Point 1: 29°34'30"E, 04°46'0"N
  Point 2: 29°16'45"E, 04°46'0"N
  Point 3: 29°16'45"E, 05°05'30"N
  Point 4: 29°34'30"E, 05°05'30"N
  Buffer: 20m


Map(center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position', 'transparent_b…

## Load SRTM 30 DEM for Lineament Analysis


In [62]:
# Load SRTM 30m DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)

# Calculate terrain derivatives for lineament detection
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
hillshade = ee.Terrain.hillshade(elevation)

# Calculate curvature for enhanced lineament detection
# Use gradient-based approach for better lineament detection
# Smoothing to reduce noise (1-2 pixels optimal: 30-60m at SRTM 30m resolution)
gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
elev_smooth = elevation.convolve(gaussian_filter).resample('bilinear')

# Calculate gradients (first derivatives)
grads = elev_smooth.gradient()
dz_dx = grads.select('x')
dz_dy = grads.select('y')

# Calculate second derivatives (curvature)
second_grads = grads.gradient()
d2z_dx2 = second_grads.select('x')
d2z_dy2 = second_grads.select('y')
total_curvature = d2z_dx2.abs().add(d2z_dy2.abs()).rename('curvature')

# Calculate aspect variance (high variance indicates linear features/lineaments)
# Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
kernel_variance = ee.Kernel.square(radius=2, units='pixels')
aspect_variance = aspect.reduceNeighborhood(
    reducer=ee.Reducer.variance(),
    kernel=kernel_variance
).rename('aspect_variance')

# Add to map
Map.addLayer(elevation, {'min': 0, 'max': 2000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'SRTM Elevation')
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['white', 'brown']}, 'Slope')
Map.addLayer(aspect, {'min': 0, 'max': 360, 'palette': ['red', 'yellow', 'green', 'cyan', 'blue', 'magenta', 'red']}, 'Aspect')
Map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade', False)
Map.addLayer(total_curvature, {'min': 0, 'max': 0.03, 'palette': ['white', 'orange', 'red', 'black']}, 'Curvature (Gradient-based)', False)
Map.addLayer(aspect_variance, {'min': 0, 'max': 500, 'palette': ['white', 'blue', 'purple']}, 'Aspect Variance', False)

print("SRTM DEM and terrain derivatives loaded successfully!")
Map


SRTM DEM and terrain derivatives loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [ ]:
# Check aspect variance statistics and adjust visualization
print("Calculating aspect variance statistics...")

# Get statistics for better visualization
aspect_var_stats = aspect_variance.reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        ee.Reducer.percentile([5, 25, 50, 75, 95]),
        sharedInputs=True
    ),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
)

# Get the statistics values
stats_dict = aspect_var_stats.getInfo()
min_val = stats_dict.get('aspect_variance_min', 0)
max_val = stats_dict.get('aspect_variance_max', 50000)
p5 = stats_dict.get('aspect_variance_p5', 0)
p25 = stats_dict.get('aspect_variance_p25', 0)
p50 = stats_dict.get('aspect_variance_p50', 0)
p75 = stats_dict.get('aspect_variance_p75', 0)
p95 = stats_dict.get('aspect_variance_p95', 5000)

print("\nAspect Variance Statistics:")
print(f"  Minimum: {min_val:.2f}")
print(f"  5th percentile: {p5:.2f}")
print(f"  25th percentile: {p25:.2f}")
print(f"  50th percentile (median): {p50:.2f}")
print(f"  75th percentile: {p75:.2f}")
print(f"  95th percentile: {p95:.2f}")
print(f"  Maximum: {max_val:.2f}")

# Update visualization with adjusted range
# Use 5th to 95th percentile to avoid outliers and see variation
print(f"\nUpdating visualization range: {p5:.0f} to {p95:.0f}")

# Remove old layer if it exists (by name)
try:
    # Try to remove the old layer
    pass  # geemap doesn't have easy layer removal, so we'll just add a new one
except:
    pass

# Add updated visualization with better range
Map.addLayer(aspect_variance, {
    'min': p5,  # Start from 5th percentile
    'max': p95,  # End at 95th percentile (shows 90% of data range)
    'palette': ['white', 'lightblue', 'blue', 'purple', 'indigo']  # Using valid CSS colors
}, 'Aspect Variance (Adjusted)', False)

print("\nVisualization updated! Now you should see:")
print("  - White/light blue: Low variance areas (uniform terrain)")
print("  - Blue: Moderate variance (some structural variation)")
print("  - Purple: High variance (structural features/lineaments)")
print("  - Dark purple: Very high variance (major structural features)")

Map


## Load PALSAR Data - Period 1 (2001-2006)


In [ ]:
# Diagnostic: Check what PALSAR data is actually available
print("=" * 60)
print("PALSAR DATA AVAILABILITY DIAGNOSTIC")
print("=" * 60)

# Check all available PALSAR collections
print("\nChecking multiple PALSAR sources for 2023-2024...")

# 1. ALOS PALSAR (original)
palsar_1 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31')
count_1 = palsar_1.size().getInfo()
print(f"\n1. ALOS PALSAR (original): {count_1} images")
if count_1 > 0:
    dates_1 = palsar_1.aggregate_array('system:time_start').getInfo()
    print("   Available dates:")
    for i, date in enumerate(sorted(dates_1), 1):
        date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
        print(f"     {i}. {date_str}")

# 2. ALOS-2 PALSAR-2
palsar_2 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/YEARLY/SAR') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31')
count_2 = palsar_2.size().getInfo()
print(f"\n2. ALOS-2 PALSAR-2: {count_2} images")
if count_2 > 0:
    dates_2 = palsar_2.aggregate_array('system:time_start').getInfo()
    print("   Available dates:")
    for i, date in enumerate(sorted(dates_2), 1):
        date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
        print(f"     {i}. {date_str}")
else:
    # Show date range of all available ALOS-2 data
    all_alos2 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/YEARLY/SAR') \
        .filterBounds(study_area_buffered)
    if all_alos2.size().getInfo() > 0:
        date_min_2 = all_alos2.aggregate_min('system:time_start').getInfo()
        date_max_2 = all_alos2.aggregate_max('system:time_start').getInfo()
        earliest_2 = ee.Date(date_min_2).format('YYYY-MM-dd').getInfo()
        latest_2 = ee.Date(date_max_2).format('YYYY-MM-dd').getInfo()
        print(f"   Date range available: {earliest_2} to {latest_2}")

# 3. ALOS-4 PALSAR-3 (try different possible collection names)
palsar_3a = None
count_3a = 0
try:
    palsar_3a = ee.ImageCollection('JAXA/ALOS/PALSAR-3/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .filterDate('2023-01-01', '2024-12-31')
    count_3a = palsar_3a.size().getInfo()
    print(f"\n3a. ALOS-4 PALSAR-3 (JAXA/ALOS/PALSAR-3/YEARLY/SAR): {count_3a} images")
    if count_3a > 0:
        dates_3a = palsar_3a.aggregate_array('system:time_start').getInfo()
        print("   Available dates:")
        for i, date in enumerate(sorted(dates_3a), 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"     {i}. {date_str}")
except Exception as e:
    print(f"3a. ALOS-4 PALSAR-3 (JAXA/ALOS/PALSAR-3/YEARLY/SAR): Collection not found or error")

# Try alternative collection name for PALSAR-3
palsar_3b = None
count_3b = 0
try:
    palsar_3b = ee.ImageCollection('JAXA/ALOS-4/PALSAR-3/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .filterDate('2023-01-01', '2024-12-31')
    count_3b = palsar_3b.size().getInfo()
    print(f"\n3b. ALOS-4 PALSAR-3 (JAXA/ALOS-4/PALSAR-3/YEARLY/SAR): {count_3b} images")
    if count_3b > 0:
        dates_3b = palsar_3b.aggregate_array('system:time_start').getInfo()
        print("   Available dates:")
        for i, date in enumerate(sorted(dates_3b), 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"     {i}. {date_str}")
except Exception as e:
    print(f"3b. ALOS-4 PALSAR-3 (JAXA/ALOS-4/PALSAR-3/YEARLY/SAR): Collection not found or error")

# Determine which collection to use
selected_collection = None
selected_name = None
selected_dates = []

if count_2 > 0:
    selected_collection = palsar_2
    selected_name = "ALOS-2 PALSAR-2"
    selected_dates = sorted(palsar_2.aggregate_array('system:time_start').getInfo())
    print(f"\n✓ Selected: {selected_name} ({count_2} images)")
elif count_3a > 0:
    selected_collection = palsar_3a
    selected_name = "ALOS-4 PALSAR-3"
    selected_dates = sorted(palsar_3a.aggregate_array('system:time_start').getInfo())
    print(f"\n✓ Selected: {selected_name} ({count_3a} images)")
elif count_3b > 0:
    selected_collection = palsar_3b
    selected_name = "ALOS-4 PALSAR-3"
    selected_dates = sorted(palsar_3b.aggregate_array('system:time_start').getInfo())
    print(f"\n✓ Selected: {selected_name} ({count_3b} images)")
elif count_1 > 0:
    selected_collection = palsar_1
    selected_name = "ALOS PALSAR (original)"
    selected_dates = sorted(palsar_1.aggregate_array('system:time_start').getInfo())
    print(f"\n✓ Selected: {selected_name} ({count_1} images)")
else:
    print("\n⚠️  No 2023-2024 data found in any PALSAR collection")
    print("   Will use latest available data as fallback")
    
    # Show what's available in each collection
    print("\n   Checking latest available dates in each collection:")
    
    # ALOS-2 latest
    latest_alos2 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .sort('system:time_start', False) \
        .limit(3)
    if latest_alos2.size().getInfo() > 0:
        dates_alos2 = latest_alos2.aggregate_array('system:time_start').getInfo()
        print(f"   ALOS-2 PALSAR-2 latest dates:")
        for i, date in enumerate(sorted(dates_alos2, reverse=True), 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"     {i}. {date_str}")
    
    # Original PALSAR latest
    latest_palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .sort('system:time_start', False) \
        .limit(3)
    if latest_palsar.size().getInfo() > 0:
        dates_palsar = latest_palsar.aggregate_array('system:time_start').getInfo()
        print(f"   ALOS PALSAR (original) latest dates:")
        for i, date in enumerate(sorted(dates_palsar, reverse=True), 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"     {i}. {date_str}")

# If we found data, show dates
if selected_collection is not None and len(selected_dates) > 0:
    print(f"\n📅 Dates available in {selected_name}:")
    for i, date in enumerate(selected_dates, 1):
        date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
        print(f"   {i}. {date_str}")
    
    earliest = ee.Date(selected_dates[0]).format('YYYY-MM-dd').getInfo()
    latest = ee.Date(selected_dates[-1]).format('YYYY-MM-dd').getInfo()
    print(f"\n   Date range: {earliest} to {latest}")

print("=" * 60)


In [63]:
# Load PALSAR data for period 2001-2006
# Note: PALSAR data availability may vary. Using JAXA ALOS PALSAR collection
palsar_collection_2001_2006 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
    .filterBounds(study_area_buffered) \
    .filterDate('2007-01-01', '2011-12-31')  # PALSAR data typically starts from 2007

# Alternative: Use ALOS PALSAR L1.0 collection if available
# Check available years and select appropriate time range
print("Checking PALSAR collection availability...")
print(f"Collection size: {palsar_collection_2001_2006.size().getInfo()}")

# If no data in 2001-2006, use earliest available data
if palsar_collection_2001_2006.size().getInfo() == 0:
    print("No PALSAR data for 2001-2006, using earliest available data...")
    palsar_collection_2001_2006 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .sort('system:time_start', True) \
        .limit(5)

# Create median composite
palsar_2001_2006 = palsar_collection_2001_2006.median().clip(study_area_buffered)

# PALSAR bands: HH (horizontal-horizontal) and HV (horizontal-vertical)
# Convert to dB scale for better visualization
palsar_2001_2006_db = palsar_2001_2006.select(['HH', 'HV']).multiply(ee.Image.constant(0.1)).log10().multiply(10)

# Add to map
Map.addLayer(palsar_2001_2006_db.select('HH'), {
    'min': -25,
    'max': 5,
    'palette': ['black', 'white']
}, 'PALSAR HH (2001-2006)', False)

Map.addLayer(palsar_2001_2006_db.select('HV'), {
    'min': -30,
    'max': 0,
    'palette': ['black', 'white']
}, 'PALSAR HV (2001-2006)', False)

# Create RGB composite (HH, HV, HH/HV ratio)
hh_hv_ratio = palsar_2001_2006_db.select('HH').divide(palsar_2001_2006_db.select('HV').add(0.01))
palsar_rgb_2001_2006 = ee.Image.cat([
    palsar_2001_2006_db.select('HH'),
    palsar_2001_2006_db.select('HV'),
    hh_hv_ratio
]).rename(['R', 'G', 'B'])

Map.addLayer(palsar_rgb_2001_2006, {
    'min': -25,
    'max': 5
}, 'PALSAR RGB Composite (2001-2006)')

print("PALSAR data (2001-2006) loaded successfully!")
Map


Checking PALSAR collection availability...
Collection size: 4
PALSAR data (2001-2006) loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [ ]:
# Load PALSAR data for period 2023-2024
# Check multiple sources: ALOS PALSAR, ALOS-2 PALSAR-2, ALOS-4 PALSAR-3

print("=" * 60)
print("LOADING PALSAR DATA FOR 2023-2024")
print("=" * 60)

# Try ALOS-2 PALSAR-2 first (most likely to have recent data)
palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/YEARLY/SAR') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31')

collection_size = palsar_collection_2023_2024.size().getInfo()
data_source = "ALOS-2 PALSAR-2"

# If no ALOS-2 data, try ALOS-4 PALSAR-3
if collection_size == 0:
    print(f"No data in {data_source}, trying ALOS-4 PALSAR-3...")
    try:
        palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR-3/YEARLY/SAR') \
            .filterBounds(study_area_buffered) \
            .filterDate('2023-01-01', '2024-12-31')
        collection_size = palsar_collection_2023_2024.size().getInfo()
        data_source = "ALOS-4 PALSAR-3"
    except:
        # Try alternative collection name
        try:
            palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS-4/PALSAR-3/YEARLY/SAR') \
                .filterBounds(study_area_buffered) \
                .filterDate('2023-01-01', '2024-12-31')
            collection_size = palsar_collection_2023_2024.size().getInfo()
            data_source = "ALOS-4 PALSAR-3"
        except:
            print("ALOS-4 PALSAR-3 collection not found, trying original ALOS PALSAR...")
            palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
                .filterBounds(study_area_buffered) \
                .filterDate('2023-01-01', '2024-12-31')
            collection_size = palsar_collection_2023_2024.size().getInfo()
            data_source = "ALOS PALSAR (original)"

print(f"\nCollection: {data_source}")
print(f"Images found for 2023-2024: {collection_size}")

# Show dates if data found
if collection_size > 0:
    dates = palsar_collection_2023_2024.aggregate_array('system:time_start').getInfo()
    sorted_dates = sorted(dates)
    print(f"\n📅 Available image dates:")
    for i, date in enumerate(sorted_dates, 1):
        date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
        print(f"   {i}. {date_str}")
    
    earliest = ee.Date(sorted_dates[0]).format('YYYY-MM-dd').getInfo()
    latest = ee.Date(sorted_dates[-1]).format('YYYY-MM-dd').getInfo()
    print(f"\n   Date range: {earliest} to {latest}")

# If no data, use latest available from any source
actual_period_label = "2023-2024"
if collection_size == 0:
    print(f"\n⚠️  No {data_source} data for 2023-2024, checking for latest available data...")
    
    # Try to get latest from ALOS-2 first
    latest_alos2 = ee.ImageCollection('JAXA/ALOS/PALSAR-2/YEARLY/SAR') \
        .filterBounds(study_area_buffered) \
        .sort('system:time_start', False) \
        .limit(5)
    
    if latest_alos2.size().getInfo() > 0:
        palsar_collection_2023_2024 = latest_alos2
        data_source = "ALOS-2 PALSAR-2 (latest available)"
    else:
        # Fall back to original PALSAR
        palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR') \
            .filterBounds(study_area_buffered) \
            .sort('system:time_start', False) \
            .limit(5)
        data_source = "ALOS PALSAR (latest available)"
    
    # Get actual dates being used
    actual_dates = palsar_collection_2023_2024.aggregate_array('system:time_start').getInfo()
    if len(actual_dates) > 0:
        sorted_actual_dates = sorted(actual_dates)
        earliest_actual = ee.Date(sorted_actual_dates[0]).format('YYYY-MM-dd').getInfo()
        latest_actual = ee.Date(sorted_actual_dates[-1]).format('YYYY-MM-dd').getInfo()
        actual_period_label = f"{earliest_actual} to {latest_actual} (Latest Available)"
        print(f"\n📅 Using fallback data from {data_source}:")
        for i, date in enumerate(sorted_actual_dates, 1):
            date_str = ee.Date(date).format('YYYY-MM-dd').getInfo()
            print(f"   {i}. {date_str}")
        print(f"\n   Date range: {earliest_actual} to {latest_actual}")

# Create median composite
palsar_2023_2024 = palsar_collection_2023_2024.median().clip(study_area_buffered)

# Convert to dB scale
# Note: PALSAR-2 and PALSAR-3 may have different scaling than PALSAR-1
palsar_2023_2024_db = palsar_2023_2024.select(['HH', 'HV']).multiply(ee.Image.constant(0.1)).log10().multiply(10)

# Add to map
Map.addLayer(palsar_2023_2024_db.select('HH'), {
    'min': -25,
    'max': 5,
    'palette': ['black', 'white']
}, f'PALSAR HH ({actual_period_label})', False)

Map.addLayer(palsar_2023_2024_db.select('HV'), {
    'min': -30,
    'max': 0,
    'palette': ['black', 'white']
}, f'PALSAR HV ({actual_period_label})', False)

# Create RGB composite
hh_hv_ratio_2023 = palsar_2023_2024_db.select('HH').divide(palsar_2023_2024_db.select('HV').add(0.01))
palsar_rgb_2023_2024 = ee.Image.cat([
    palsar_2023_2024_db.select('HH'),
    palsar_2023_2024_db.select('HV'),
    hh_hv_ratio_2023
]).rename(['R', 'G', 'B'])

Map.addLayer(palsar_rgb_2023_2024, {
    'min': -25,
    'max': 5
}, f'PALSAR RGB Composite ({actual_period_label})')

print(f"\n✓ PALSAR data loaded successfully!")
print(f"  Data source: {data_source}")
print(f"  Period label: {actual_period_label}")
if collection_size == 0:
    print(f"  ⚠️  Note: This is fallback data, not actual 2023-2024 data")
print("=" * 60)
Map


Collection size: 0
No PALSAR data for 2023-2024, using latest available data...
PALSAR data (2023-2024) loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [66]:
# Lineament extraction using DEM-based methods
# Method 1: Edge detection on hillshade
def extract_lineaments_dem(elevation_img, slope_img, aspect_img, hillshade_img):
    """Extract lineaments from DEM using multiple methods"""
    
    # Method 1: Slope-based lineaments (steep slopes often indicate faults)
    slope_threshold = slope_img.gt(15)  # Threshold for steep slopes
    
    # Method 2: Aspect-based lineaments (linear features in aspect)
    # Convert aspect to radians and apply edge detection
    aspect_rad = aspect_img.multiply(ee.Image.constant(3.14159)).divide(180)
    
    # Method 3: Hillshade edge detection
    # Apply Sobel filter for edge detection
    # Create kernel for edge detection
    kernel = ee.Kernel.fixed(3, 3, [
        [-1, -1, -1],
        [0, 0, 0],
        [1, 1, 1]
    ])
    
    # Apply convolution for edge detection
    edges_x = hillshade_img.convolve(kernel)
    edges_y = hillshade_img.convolve(kernel.rotate(90))
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Threshold edges to identify lineaments
    lineament_edges = edges.gt(30).rename('Lineament_Edges')
    
    # Method 4: Curvature-based lineaments (gradient approach)
    # Smooth elevation first to reduce noise
    gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
    elev_smooth = elevation_img.convolve(gaussian_filter).resample('bilinear')
    
    # Calculate gradients and second derivatives
    grads = elev_smooth.gradient()
    second_grads = grads.gradient()
    d2z_dx2 = second_grads.select('x')
    d2z_dy2 = second_grads.select('y')
    curvature = d2z_dx2.abs().add(d2z_dy2.abs())
    
    # Threshold based on curvature magnitude (adjust threshold as needed)
    lineament_curvature = curvature.gt(0.02).rename('Lineament_Curvature')
    
    # Method 5: Aspect variance lineaments (high variance = linear features)
    # Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
    kernel_variance = ee.Kernel.square(radius=2, units='pixels')
    aspect_var = aspect_img.reduceNeighborhood(
        reducer=ee.Reducer.variance(),
        kernel=kernel_variance
    ).rename('aspect_variance')
    # Use a fixed threshold for aspect variance
    # For aspect (0-360 degrees), variance with 2-pixel radius typically ranges 0-50000
    # High variance (>10000) indicates linear features/lineaments
    # Adjust this threshold based on your study area characteristics
    aspect_var_threshold = ee.Image.constant(10000)  # Adjust as needed (typical range: 5000-20000)
    lineament_aspect_var = aspect_var.gt(aspect_var_threshold).rename('Lineament_Aspect_Var')
    
    # Combine all DEM-based methods
    lineament_combined = lineament_edges.add(lineament_curvature).add(lineament_aspect_var).gt(0).rename('Lineament_DEM')
    
    return {
        'lineament_edges': lineament_edges,
        'lineament_curvature': lineament_curvature,
        'lineament_aspect_var': lineament_aspect_var,
        'lineament_combined': lineament_combined,
        'edges': edges
    }

# Extract lineaments from DEM
dem_lineaments = extract_lineaments_dem(elevation, slope, aspect, hillshade)

# Visualize DEM-based lineaments
Map.addLayer(dem_lineaments['edges'], {
    'min': 0,
    'max': 100,
    'palette': ['black', 'white']
}, 'DEM Edge Detection', False)

Map.addLayer(dem_lineaments['lineament_edges'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from Edges (DEM)')

Map.addLayer(dem_lineaments['lineament_curvature'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'cyan']
}, 'Lineaments from Curvature (DEM)', False)

Map.addLayer(dem_lineaments['lineament_combined'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (DEM)')

print("DEM-based lineaments extracted!")
Map


DEM-based lineaments extracted!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [67]:
# Lineament extraction from PALSAR radar data
# Radar backscatter can reveal structural features

def extract_lineaments_palsar(palsar_img):
    """Extract lineaments from PALSAR data"""
    
    # Use HH polarization for lineament detection (better for structural features)
    hh = palsar_img.select('HH')
    
    # Apply edge detection using Sobel filter
    kernel_x = ee.Kernel.fixed(3, 3, [
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    
    kernel_y = ee.Kernel.fixed(3, 3, [
        [-1, -2, -1],
        [0, 0, 0],
        [1, 2, 1]
    ])
    
    # Apply convolution
    edges_x = hh.convolve(kernel_x)
    edges_y = hh.convolve(kernel_y)
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Method 2: HH/HV ratio (enhanced lineament detection)
    # Ratio helps identify structural features
    hh_hv_ratio = palsar_img.select('HH').divide(palsar_img.select('HV').add(0.01)).rename('HH_HV_Ratio')
    
    # Edge detection on ratio
    ratio_edges_x = hh_hv_ratio.convolve(kernel_x)
    ratio_edges_y = hh_hv_ratio.convolve(kernel_y)
    ratio_edges = ratio_edges_x.pow(2).add(ratio_edges_y.pow(2)).sqrt()
    
    # Combine HH edges and ratio edges for better detection
    combined_edges = edges.add(ratio_edges)
    
    # Threshold to identify lineaments using fixed threshold
    # PALSAR edge values typically range from 0-50 for combined edges
    # Use a threshold based on typical values (adjust as needed)
    # For 75th percentile equivalent, use ~30-40% of max range
    edge_threshold = ee.Image.constant(20)  # Adjust based on your data (typical range: 15-30)
    lineament_threshold = combined_edges.gt(edge_threshold).rename('Lineament_PALSAR')
    
    return {
        'edges': combined_edges,
        'lineament_threshold': lineament_threshold,
        'hh_hv_ratio': hh_hv_ratio
    }

# Extract lineaments from both periods
palsar_lineaments_2001_2006 = extract_lineaments_palsar(palsar_2001_2006_db)
palsar_lineaments_2023_2024 = extract_lineaments_palsar(palsar_2023_2024_db)

# Visualize PALSAR-based lineaments
Map.addLayer(palsar_lineaments_2001_2006['edges'], {
    'min': 0,
    'max': 50,
    'palette': ['black', 'white']
}, 'PALSAR Edges (2001-2006)', False)

Map.addLayer(palsar_lineaments_2001_2006['lineament_threshold'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from PALSAR (2001-2006)')

Map.addLayer(palsar_lineaments_2023_2024['lineament_threshold'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from PALSAR (2023-2024)', False)

# Visualize HH/HV ratio for structural analysis
Map.addLayer(palsar_lineaments_2001_2006['hh_hv_ratio'], {
    'min': 0,
    'max': 5,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'HH/HV Ratio (2001-2006)', False)

Map.addLayer(palsar_lineaments_2023_2024['hh_hv_ratio'], {
    'min': 0,
    'max': 5,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'HH/HV Ratio (2023-2024)', False)

print("PALSAR-based lineaments extracted!")
Map


PALSAR-based lineaments extracted!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

In [68]:
# Combine DEM and PALSAR lineaments for comprehensive lineament map
# Enhanced approach: DEM curvature + PALSAR edges/ratios + aspect variance
def combine_lineaments(dem_lineaments, palsar_lineaments, aspect_variance_img):
    """Combine DEM and PALSAR lineaments with enhanced confidence"""
    
    # Extract individual components
    dem_curvature = dem_lineaments['lineament_curvature'].unmask(0)
    dem_aspect_var = dem_lineaments['lineament_aspect_var'].unmask(0)
    dem_edges = dem_lineaments['lineament_edges'].unmask(0)
    palsar_edges = palsar_lineaments['edges'].unmask(0)
    palsar_threshold = palsar_lineaments['lineament_threshold'].unmask(0)
    
    # Normalize PALSAR edges to 0-1 range using unitScale (server-side operation)
    # Use percentile-based scaling to avoid outliers
    # PALSAR edges typically range from -50 to 50 for edge detection
    palsar_edges_norm = palsar_edges.unitScale(-50, 50).clamp(0, 1)
    
    # Normalize aspect variance to 0-1 range using unitScale
    # Aspect variance typically ranges from 0 to ~50000 for 2-pixel radius
    aspect_var_norm = aspect_variance_img.unitScale(0, 50000).clamp(0, 1)
    
    # Combine with weights for higher confidence detection
    # DEM curvature: 0.3, PALSAR edges: 0.3, Aspect variance: 0.2, DEM edges: 0.2
    combined_score = (
        dem_curvature.multiply(0.3)
        .add(palsar_edges_norm.multiply(0.3))
        .add(aspect_var_norm.multiply(0.2))
        .add(dem_edges.multiply(0.2))
    )
    
    # Threshold for final lineament map (adjust threshold as needed)
    combined = combined_score.gt(0.4).rename('Combined_Lineaments')
    
    # Create confidence map (0-1 scale)
    confidence = combined_score.rename('Lineament_Confidence')
    
    return combined.addBands(confidence)

# Create combined lineament maps for both periods
# Pass aspect_variance from Cell 6
combined_lineaments_2001_2006 = combine_lineaments(
    dem_lineaments,
    palsar_lineaments_2001_2006,
    aspect_variance
)

combined_lineaments_2023_2024 = combine_lineaments(
    dem_lineaments,
    palsar_lineaments_2023_2024,
    aspect_variance
)

# Visualize combined lineaments
Map.addLayer(combined_lineaments_2001_2006.select('Combined_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (2001-2006)')

Map.addLayer(combined_lineaments_2001_2006.select('Lineament_Confidence'), {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Confidence (2001-2006)', False)

Map.addLayer(combined_lineaments_2023_2024.select('Combined_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (2023-2024)', False)

Map.addLayer(combined_lineaments_2023_2024.select('Lineament_Confidence'), {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Confidence (2023-2024)', False)

print("Combined lineament maps created!")
Map


Combined lineament maps created!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Lineament Density Analysis


In [69]:
# Calculate lineament density (lineaments per unit area)
def calculate_lineament_density(lineament_img, radius=500):
    """Calculate lineament density using focal statistics"""
    
    # Create a circular kernel for the moving window
    # Convert radius from meters to pixels (assuming 30m resolution)
    radius_pixels = radius / 30
    kernel = ee.Kernel.circle(radius=radius_pixels, units='pixels')
    
    # Count lineaments in a moving window using reduceNeighborhood
    lineament_count = lineament_img.select('Combined_Lineaments').reduceNeighborhood(
        reducer=ee.Reducer.sum(),
        kernel=kernel
    )
    
    # Calculate area in square kilometers (π * r²)
    area_km2 = 3.14159 * radius * radius / 1000000
    
    # Calculate density (lineaments per km²)
    lineament_density = lineament_count.divide(ee.Image.constant(area_km2)).rename('Lineament_Density')
    
    return lineament_density

# Calculate density for both periods
lineament_density_2001_2006 = calculate_lineament_density(combined_lineaments_2001_2006)
lineament_density_2023_2024 = calculate_lineament_density(combined_lineaments_2023_2024)

# Visualize lineament density
Map.addLayer(lineament_density_2001_2006, {
    'min': 0,
    'max': 10,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Density (2001-2006)')

Map.addLayer(lineament_density_2023_2024, {
    'min': 0,
    'max': 10,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Density (2023-2024)', False)

print("Lineament density calculated!")
Map


Lineament density calculated!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Lineament Orientation Analysis


In [70]:
# Analyze lineament orientations (rose diagram data)
# Extract lineament directions from aspect and edge orientations

def analyze_lineament_orientation(lineament_img, aspect_img):
    """Analyze lineament orientations"""
    
    # Use aspect where lineaments exist
    lineament_aspect = aspect_img.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    # Classify into orientation classes (N, NE, E, SE, S, SW, W, NW)
    # Convert to 8 classes (45-degree intervals)
    orientation_classes = lineament_aspect.divide(45).int().mod(8).rename('Orientation_Class')
    
    # Create orientation map
    orientation_map = orientation_classes.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    return orientation_map

# Analyze orientations for both periods
lineament_orientation_2001_2006 = analyze_lineament_orientation(
    combined_lineaments_2001_2006,
    aspect
)

lineament_orientation_2023_2024 = analyze_lineament_orientation(
    combined_lineaments_2023_2024,
    aspect
)

# Visualize orientations
Map.addLayer(lineament_orientation_2001_2006, {
    'min': 0,
    'max': 7,
    'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'magenta']
}, 'Lineament Orientation (2001-2006)')

Map.addLayer(lineament_orientation_2023_2024, {
    'min': 0,
    'max': 7,
    'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'magenta']
}, 'Lineament Orientation (2023-2024)', False)

print("Lineament orientation analysis completed!")
print("Note: 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW")
Map


Lineament orientation analysis completed!
Note: 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

## Temporal Change Detection


In [71]:
# Compare lineaments between the two time periods
def calculate_lineament_change(period1, period2):
    """Calculate changes in lineament patterns"""
    
    # New lineaments (present in period 2 but not in period 1)
    new_lineaments = period2.select('Combined_Lineaments').subtract(
        period1.select('Combined_Lineaments')
    ).gt(0).rename('New_Lineaments')
    
    # Disappeared lineaments (present in period 1 but not in period 2)
    disappeared_lineaments = period1.select('Combined_Lineaments').subtract(
        period2.select('Combined_Lineaments')
    ).gt(0).rename('Disappeared_Lineaments')
    
    # Stable lineaments (present in both periods)
    stable_lineaments = period1.select('Combined_Lineaments').multiply(
        period2.select('Combined_Lineaments')
    ).rename('Stable_Lineaments')
    
    # Change in density
    density_change = lineament_density_2023_2024.subtract(lineament_density_2001_2006).rename('Density_Change')
    
    return ee.Image.cat([new_lineaments, disappeared_lineaments, stable_lineaments, density_change])

# Calculate changes
lineament_changes = calculate_lineament_change(
    combined_lineaments_2001_2006,
    combined_lineaments_2023_2024
)

# Visualize changes
Map.addLayer(lineament_changes.select('New_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'green']
}, 'New Lineaments (2023-2024)')

Map.addLayer(lineament_changes.select('Disappeared_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Disappeared Lineaments', False)

Map.addLayer(lineament_changes.select('Stable_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Stable Lineaments', False)

Map.addLayer(lineament_changes.select('Density_Change'), {
    'min': -5,
    'max': 5,
    'palette': ['blue', 'white', 'red']
}, 'Lineament Density Change (2023-2024 vs 2001-2006)')

print("Temporal change analysis completed!")
Map


Temporal change analysis completed!


Map(bottom=127778.0, center=[4.92951505644009, 29.426879882812504], controls=(WidgetControl(options=['position…

In [73]:
# Export lineament maps
# Note: Uncomment and run to export to Google Drive

def export_lineament_maps():
    """Export all lineament maps"""
    
    # DEM-based lineaments
    geemap.ee_export_image(
        dem_lineaments['lineament_combined'],
        filename='lineaments_dem.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # PALSAR-based lineaments
    geemap.ee_export_image(
        palsar_lineaments_2001_2006['lineament_threshold'],
        filename='lineaments_palsar_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        palsar_lineaments_2023_2024['lineament_threshold'],
        filename='lineaments_palsar_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Combined lineaments
    geemap.ee_export_image(
        combined_lineaments_2001_2006,
        filename='lineaments_combined_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        combined_lineaments_2023_2024,
        filename='lineaments_combined_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Lineament density
    geemap.ee_export_image(
        lineament_density_2001_2006,
        filename='lineament_density_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        lineament_density_2023_2024,
        filename='lineament_density_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Lineament orientation
    geemap.ee_export_image(
        lineament_orientation_2001_2006,
        filename='lineament_orientation_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        lineament_orientation_2023_2024,
        filename='lineament_orientation_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Change detection
    geemap.ee_export_image(
        lineament_changes,
        filename='lineament_changes.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

print("Export functions defined. Uncomment and call to export maps.")
print("Example: export_lineament_maps()")


Export functions defined. Uncomment and call to export maps.
Example: export_lineament_maps()
